# World Development Indicators Analysis

### Load Necessary Libraries
#### We only need two here, to perform SQL queries and handle subsequent dataframes

In [ ]:
from pandasql import sqldf
import pandas as pd


In [ ]:
df = pd.read_csv("WDI_csv/WDIData.csv")
df.head(3)

### Pull the columns needed for the this analysis

In [ ]:
q = """
    SELECT 
        "Country Name", "Indicator Name", "2000", "2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008",
        "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020"
    FROM df
    ;
    """

df = sqldf(q)
df

### Pull the Indicators we will be using. These were picked randomly

In [ ]:
q = """
    SELECT *
    FROM df
    WHERE "Indicator Name" IN 
        ("Population growth (annual %)"
        , "Urban population growth (annual %)"
        , "Rural population growth (annual %)"
        , "GDP per capita (current US$)"
        , "GDP growth (annual %)"
        , "Unemployment, total (% of total labor force) (national estimate)"
        , "Unemployment with basic education (% of total labor force with basic education)"
        , "Unemployment with basic education, male (% of male labor force with basic education)"
        , "Unemployment with basic education, female (% of female labor force with basic education)"
        , "Unemployment with intermediate education (% of total labor force with intermediate education)"
        , "Unemployment with intermediate education, male (% of male labor force with intermediate education)"
        , "Unemployment with intermediate education, female (% of female labor force with intermediate education)"
        , "Unemployment with advanced education (% of total labor force with advanced education)"
        , "Unemployment with advanced education, male (% of male labor force with advanced education)"
        , "Unemployment with advanced education, female (% of female labor force with advanced education)"
        , "Inflation, consumer prices (annual %)"
        , "Life expectancy at birth, total (years)"
        , "Electricity production from renewable sources, excluding hydroelectric (% of total)"
        , "Proportion of seats held by women in national parliaments (%)"
        , "Research and development expenditure (% of GDP)"
        , "CO2 emissions (metric tons per capita)"
        , "Individuals using the Internet (% of population)"
        , "Mobile cellular subscriptions"
        , "Number of maternal deaths"
        , "Number of infant deaths")
    ;
    """

df = sqldf(q)
df

### Create a **Continent** field and group all countries by it

In [ ]:
q = """
    SELECT *,
        CASE
            WHEN "Country Name" IN 
                ("Algeria", "Angola", "Benin", "Botswana", "Burkina Faso", "Burundi", "Cameroon", "Cape Verde", "Central African Republic", "Chad", 
                 "Camoros", "Democratic Republic of the Congo", "Republic of the Congo", "Djibouti", "Egypt", "Equatorial Guinea", "Eritrea", "Ethiopia",
                 "Gabon", "Gambia", "Ghana", "Guinea", "Guinea-Bissau", "Ivory Coast", "Kenya", "Lesotho", "Liberia","Libya", "Madagascar", "Malawi", "Mali", 
                 "Mauritania", "Mauritius", "Morocco", "Mozambique", "Namibia", "Niger", "Nigeria", "Rwanda", "Sao Tome and Principe", "Senegal", "Seychelles", 
                 "Sierra Leone", "Somalia", "South Africa", "South Sudan", "Sudan", "Swaziland", "Tanzania", "Togo", "Tunisia", "Uganda", "Zambia", "Zimbabwe")
                THEN "Africa"
            WHEN "Country Name" IN 
                ("Antigua and Barbuda", "Argentina", "Bahamas", "Barbados", "Belize", "Bolivia", "Brazil", "Canada", "Chile", "Colombia", "Costa Rica", "Cuba",
                 "Dominica", "Dominican Republic", "Ecuador", "El Salvador", "Grenada", "Guatemala", "Guyana", "Haiti", "Honduras", "Jamaica", "Mexico", "Nicaragua",
                 "Panama", "Paraguay", "Peru", "Saint Kitts and Nevis", "Saint Lucia", "Saint Vincent and the Grenadines", "Suriname", "Trinidad and Tobago", 
                 "United States", "Uruguay", "Venezuela")
                THEN "America"
            WHEN "Country Name" IN 
                ("Albania", "Andorra", "Armenia", "Austria", "Belarus", "Belgium", "Bosnia and Herzegovina", "Bulgaria", "Croatia", "Cyprus", "Czech Republic",
                 "Denmark", "Estonia", "Finland", "France", "Georgia", "Germany", "Greece", "Hungary", "Iceland", "Ireland", "Italy", "Latvia", "Liechtenstein",
                 "Lithuania", "Luxembourg", "Macedonia", "Malta", "Moldova", "Monaco", "Montenegro", "Norway", "Poland", "Portugal", "Romania", "Russia", "San Marino",
                 "Serbia", "Slovakia", "Slovenia", "Spain", "Sweden", "Switzerland", "The Netherlands", "Turkey", "Ukraine", "United Kingdom", "Vatican City")
                THEN "Europe"
            WHEN "Country Name" IN 
                ("Afghanistan", "Azerbaijan", "Bahrain", "Bangladesh", "Bhutan", "Brunei", "Cambodia", "China", "East Timor", "India", "Indonesia", "Iran", "Iraq",
                 "Israel", "Japan", "Jordan", "Kazakhstan", "Kuwait", "Kyrgyzstan", "Laos", "Lebanon", "Malaysia", "Maldives", "Mongolia", "Myanmar", "Nepal", 
                 "North Korea", "Oman", "Pakistan", "Philippines", "Qatar", "Saudi Arabia", "Singapore", "South Korea", "Sri Lanka", "Syria", "Tajikistan", "Thailand", 
                 "Turkmenistan", "United Arab Emirates", "Uzbekistan", "Vietnam", "Yemen")
                THEN "Asia"
            WHEN "Country Name" IN 
                ("Australia", "Micronesia", "Fiji", "Kiribati", "Marshall Islands", "Nauru", "New Zeland", "Palau", "Papua New Guinea", 
                 "Samoa", "Solomon Islands", "Tonga", "Tuvalu", "Vanuatu")
                THEN "Oceania"
        ELSE
            "Alien Country"
        END AS "Continent"
    FROM df
    ;
    """

df = sqldf(q)
df

### Remove Territories

In [ ]:
q = """
    SELECT *
    FROM df
    WHERE 
        "Continent" IS NOT "Alien Country"
    ;
    """

df = sqldf(q)
df

### Pull G7 Countries records

In [ ]:
q = """
    SELECT *
    FROM df
    WHERE
        "Indicator Name" IN ("Population growth (annual %)", "Life expectancy at birth, total (years)", "GDP per capita (current US$)", 
                            "Unemployment, total (% of total labor force) (national estimate)", "Inflation, consumer prices (annual %)") AND
        "Country Name" IN ("Canada", "France", "Germany", "Italy", "Japan", "", "United Kigndom", "United States")
        
    ;
    """

g7_popGrowth = sqldf(q)
g7_popGrowth.head()

### Prepare the data into a format to be used for creating our dashboard

In [ ]:
def dataframeMelt(dataframe: pd.DataFrame, val_name: str):
    """Accepts a wide dataframe and transforms it into a longer dataframe.
    Args:
        dataframe: wide dataframe to be transformed
        val_name:  Name of interest (in this case, the Indicator Name).
    Returns:
        melted (long) dataframe
    
    """
    melted_df = pd.melt(dataframe, 
                        id_vars=["Country Name", "Continent"],
                        value_vars=["2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020"],
                       var_name="Year",
                       value_name=val_name).rename(columns={"Country Name": "Country"})

    return melted_df

### Pull dataframes based on Indicator Name
PopGrowth = g7_popGrowth[g7_popGrowth["Indicator Name"] == "Population growth (annual %)"]
LifeExp = g7_popGrowth[g7_popGrowth["Indicator Name"] == "Life expectancy at birth, total (years)"]
GDPPerCapita = g7_popGrowth[g7_popGrowth["Indicator Name"] == "GDP per capita (current US$)"]
Unemploy = g7_popGrowth[g7_popGrowth["Indicator Name"] == "Unemployment, total (% of total labor force) (national estimate)"]
Inflation = g7_popGrowth[g7_popGrowth["Indicator Name"] == "Inflation, consumer prices (annual %)"]

### Convert dataframes from wide to long
PopGrowth = dataframeMelt(dataframe=PopGrowth, val_name="PopGrowth")
GDPPerCapita = dataframeMelt(dataframe=GDPPerCapita, val_name="GDPPerCapita")
Unemploy = dataframeMelt(dataframe=Unemploy, val_name="Unemploy")
Inflation = dataframeMelt(dataframe=Inflation, val_name="Inflation")
LifeExp = dataframeMelt(dataframe=LifeExp, val_name="LifeExp")


### Join the dataframes into one
wdi_Df = pd.merge(
            PopGrowth, GDPPerCapita, on=["Country", "Continent", "Year"]).merge(
            Unemploy, on=["Country", "Continent", "Year"]).merge(
            Inflation, on=["Country", "Continent", "Year"]).merge(
            LifeExp, on=["Country", "Continent", "Year"])

wdi_Df.to_csv("g7Countries.csv", index=False)
wdi_Df

[Link to dashboard of Sample G7 Countries Development Indicators](https://datastudio.google.com/reporting/03376bef-7812-4d98-be89-7d1471cbedce)